In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

gene_set_x_sample = pd.read_csv(PATH["gene_set_x_sample.tsv"], sep="\t", index_col=0)

In [ ]:
make_match_panel_keyword_arguments = dict(
    target_ascending=SETTING["target_ascending"],
    n_job=SETTING["n_job"],
    n_sampling=SETTING["n_sampling"],
    n_permutation=SETTING["n_permutation"],
    target_type=SETTING["target_type"],
    plot_std=SETTING["plot_std"],
)

for target_name, target_values in target_x_sample.iterrows():

    output_directory_path = "{}/{}".format(
        PATH["differentially_expressed_gene_set/"], target_name
    )

    ccal.establish_path(output_directory_path, "directory")

    target_values = target_values[target_values != -1]

    score_moe_p_value_fdr = ccal.make_match_panel(
        target_values,
        gene_set_x_sample,
        n_extreme=SETTING["n_extreme"],
        title="All",
        file_path_prefix="{}/all".format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )

    for gene_set_file_path in PATH["gene_set_file_paths"]:

        gene_set_name = gene_set_file_path.split("/")[-1]

        gmt = ccal.read_gmt(gene_set_file_path)

        score_moe_p_value_fdr_ = ccal.make_match_panel(
            target_values,
            gene_set_x_sample.reindex(index=gmt.index),
            score_moe_p_value_fdr=score_moe_p_value_fdr,
            n_extreme=SETTING["n_extreme"],
            title=gene_set_name,
            file_path_prefix="{}/{}".format(output_directory_path, gene_set_name),
            **make_match_panel_keyword_arguments,
        )

    score_moe_p_value_fdr_ = ccal.make_match_panel(
        target_values,
        gene_set_x_sample.reindex(index=SETTING["gene_sets_to_peek"]),
        score_moe_p_value_fdr=score_moe_p_value_fdr,
        n_extreme=None,
        title="peek",
        file_path_prefix="{}/peek".format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )